In [3]:
import pandas as pd
import numpy as np
import json
import os
import re
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_curve, auc, confusion_matrix
from sklearn.pipeline import Pipeline
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import wordnet
import string

a)remove all duplicates from sheet in excel using DATA>remove duplicates
b)using excel concatenate function joined "AgentCorrected_CatName_Primary" and "AgentCorrected_Integer_Primary" column with comma and new target column name is "primary".


In [4]:
df2_cicd=pd.read_csv('../data/TaxML-CICD - Prod_Data_0501.csv')
df2_cicd.drop_duplicates(inplace=True)
df2_cicd.dropna(how='all',inplace=True)
df2 = df2_cicd

In [5]:
df2_cicd.shape

(13918, 31)

In [6]:
def preprocess_text(message):

    #new_stopwords=['grocery']
    stpwrd = nltk.corpus.stopwords.words('english')
    #stpwrd.extend(new_stopwords)
    # 1. Init Lemmatizer
    lemmatizer = WordNetLemmatizer()
    #lowering and removing punctuation
    message = re.sub(r'[^\w\s]','', message.lower())
    #removing the numerical values and working only with text values
    message = re.sub('[^a-zA-Z]', " ", message )
    #removing the stopwords
    message = ' '.join([word for word in message.split() if word not in stpwrd and len(word)>1])
    #lemmatizing the text
    message =  " ".join([lemmatizer.lemmatize(w) for w in nltk.word_tokenize(message) if w not in string.punctuation])
    #print("message is : ",message)
    
    return message

In [7]:
def process_secondary_catname(category):
    
    if category != None:
    
        if category in ['TEMP_HEATED','TEMP_COLD','TEMP_UNHEATED']:
            category= None
    
    return category
    
def process_secondary_integer(secondary_integer):
    
    if secondary_integer != None:
#         print(secondary_integer)
        try:
            secondary_integer = str(int(float(secondary_integer)))
            if (len(secondary_integer)==3 and secondary_integer not in ['105','107']) or secondary_integer == '1' or secondary_integer == '2':      
                secondary_integer = None
        except:
            secondary_integer = None

    return secondary_integer
        

In [8]:
def process_tertiary_catname(category):
    
    if category != None:
    
        if category not in ['TEMP_HEATED','TEMP_COLD','TEMP_UNHEATED']:
            category= None
    
    return category
    
def process_tertiary_integer(secondary_integer):
    
    if secondary_integer != None:
        try:
            secondary_integer = str(int(float(secondary_integer)))
            if (len(secondary_integer))!=1:      
                secondary_integer = None
        except:
            secondary_integer = None

    return secondary_integer
        

preprocessing dataframe to get primary, secondary and tertiary columns

In [9]:
df2['AgentCorrected_Integer_Primary'] = df2['AgentCorrected_Integer_Primary'].astype('int').astype('str')

df2['primary'] = df2['AgentCorrected_CatName_Primary'] + ',' + df2['AgentCorrected_Integer_Primary']

df2['cleaned_secondary_integer'] = df2['AgentCorrected_Integer_Secondary'].apply(lambda x : process_secondary_integer(x))

df2['cleaned_secondary_catname'] = df2['AgentCorrected_CatName_Secondary'].apply(lambda x : process_secondary_catname(x))

df2.loc[df2['cleaned_secondary_integer'].isnull(),'cleaned_secondary_catname' ] = None

df2['new_secondary_catname'] = df2['cleaned_secondary_catname'].fillna(df2['AgentCorrected_CatName_Primary'])

df2['new_secondary_integer'] = df2['cleaned_secondary_integer'].fillna(df2['AgentCorrected_Integer_Primary'])

df2['new_secondary_integer'] = df2['new_secondary_integer'].astype(int).astype(str)

df2['secondary'] = df2['new_secondary_catname'] + ','+ df2['new_secondary_integer']

df2['cleaned_tertiary_integer'] = df2['AgentCorrected_Integer_Secondary'].apply(lambda x : process_tertiary_integer(x))

df2['cleaned_tertiary_integer_col2'] = df2['AgentCorrected_CatName_Secondary2.1'].apply(lambda x : process_tertiary_integer(x))

df2['cleaned_tertiary_integer'] = df2['cleaned_tertiary_integer'].fillna(df2['cleaned_tertiary_integer_col2'])

df2['cleaned_tertiary_integer'] = df2['cleaned_tertiary_integer'].fillna(df2['AgentCorrected_Integer_Primary'])

df2['cleaned_tertiary_catname'] = df2['AgentCorrected_CatName_Secondary'].apply(lambda x : process_tertiary_catname(x))

df2['cleaned_tertiary_catname_col2'] = df2['AgentCorrected_CatName_Secondary2'].apply(lambda x : process_tertiary_catname(x))

df2['cleaned_tertiary_catname'] = df2['cleaned_tertiary_catname'].fillna(df2['cleaned_tertiary_catname_col2'])

df2['cleaned_tertiary_catname'] = df2['cleaned_tertiary_catname'].fillna(df2['AgentCorrected_CatName_Primary'])

df2['tertiary'] = df2['cleaned_tertiary_catname'] + ','+ df2['cleaned_tertiary_integer']

In [10]:
df = pd.read_csv('../data/jina_final.csv', encoding='utf8',engine='python',usecols=['Item','Description','establishment_type','primary','secondary','tertiary'])
df1=df.sample(frac=1, random_state=42)
df1 = df1.fillna('')
df1['input_str'] = df1[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df1['cleanText']=df1['input_str'].map(lambda s:preprocess_text(s)) 
df1 = df1.reset_index(drop=True)
X=df1[['Item','Description','establishment_type','cleanText']]
Y_primary=df1['primary']
Y_secondary=df1['secondary']
Y_tertiary=df1['tertiary']
X_train, X_test, y_train_primary, y_test_primary,y_train_secondary, y_test_secondary,y_train_tertiary, y_test_tertiary=train_test_split(X,Y_primary,Y_secondary,Y_tertiary,test_size = .20,random_state = 42)

ValueError: Usecols do not match columns, columns expected but not found: ['tertiary']

In [11]:
df2['input_str'] = df2[['Item','Description','establishment_type']].apply(lambda x: ' '.join(x[x.notnull()]), axis = 1)
df2['cleanText']=df2['input_str'].map(lambda s:preprocess_text(s))

In [ ]:
X_cicd=df2[['Item','Description','establishment_type','cleanText']]
Y_primary_cicd=df2['primary']
Y_secondary_cicd=df2['secondary']
Y_tertiary_cicd=df2['tertiary']

X_train_cicd, X_test_cicd, y_train_primary_cicd, y_test_primary_cicd,y_train_secondary_cicd, y_test_secondary_cicd,y_train_tertiary_cicd, y_test_tertiary_cicd = train_test_split(X_cicd,
                                                    Y_primary_cicd,Y_secondary_cicd,Y_tertiary_cicd,
                                                    test_size = .20, 
                                                    random_state = 42
                                                    )
X_train_final_1 = X_train.append(X_train_cicd)
X_test_final_1 = X_test.append(X_test_cicd)
X_train_final=X_train['cleanText'].append(X_train_cicd['cleanText'])
X_test_final=X_test['cleanText'].append(X_test_cicd['cleanText'])
######primary model#######
y_train_final_primary=y_train_primary.append(y_train_primary_cicd)
y_test_final_primary=y_test_primary.append(y_test_primary_cicd)
######secondary model#######
y_train_final_secondary=y_train_secondary.append(y_train_secondary_cicd)
y_test_final_secondary=y_test_secondary.append(y_test_secondary_cicd)
######tertiary model#######
y_train_final_tertiary=y_train_tertiary.append(y_train_tertiary_cicd)
y_test_final_tertiary=y_test_tertiary.append(y_test_tertiary_cicd)

In [ ]:
import datetime
file_name='df_traintestdata_{}'.format(datetime.datetime.now())
train_size = 0.8
train_end = int(len(df1)*train_size)
df_train = df1[:train_end]
df_test = df1[train_end:]
train_end_cicd = int(len(df2)*train_size)
df2_train = df2[:train_end_cicd]
df2_test = df2[train_end_cicd:]
df2_train = df2_train[['Item','Description','establishment_type','primary','secondary','tertiary','input_str','cleanText']]
df2_test = df2_test[['Item','Description','establishment_type','primary','secondary','tertiary','input_str','cleanText']]
X_train_save = df_train.append(df2_train)
X_test_save = df_test.append(df2_test)
X_train_save['label'] = 'train'
X_test_save['label'] = 'test'
X_data = X_train_save.append(X_test_save)
X_data.to_csv(file_name)

####primary model retraining#####

In [ ]:
result=X_test_final_1
rf1 = Pipeline([('vect', CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', stop_words='english',max_df=0.85)),
       ('tfidf', TfidfTransformer()),
       ('clf', RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42,n_estimators=500))])
rf1.fit(X_train_final, y_train_final_primary)
y_pred= rf1.predict(X_test_final)
result['original_cat_primary']=y_test_final_primary
result['prediction_cat_primary']=y_pred
result['prediction_cat_primary_confscore']=rf1.predict_proba(X_test_final).max()
output={'accuracy':accuracy_score(y_pred,y_test_final_primary),'precision_score':precision_score(y_pred,y_test_final_primary,average='macro'),'recall_score':recall_score(y_pred,y_test_final_primary,average='macro')
,'f1_score':f1_score(y_pred,y_test_final_primary,average='macro')}
result['confusion_matrix_primary']=str(output)


In [ ]:
rf2 = Pipeline([('vect', CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', stop_words='english',max_df=0.85)),
       ('tfidf', TfidfTransformer()),
       ('clf', RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42,n_estimators=500))])
rf2.fit(X_train_final, y_train_final_secondary)
y_pred= rf2.predict(X_test_final)
result['original_cat_secondary']=y_test_final_secondary
result['prediction_cat_secondary']=y_pred
result['prediction_cat_secondary_confscore']=rf2.predict_proba(X_test_final).max()
output={'accuracy':accuracy_score(y_pred,y_test_final_secondary),'precision_score':precision_score(y_pred,y_test_final_secondary,average='macro'),'recall_score':recall_score(y_pred,y_test_final_secondary,average='macro')
,'f1_score':f1_score(y_pred,y_test_final_secondary,average='macro')}
result['confusion_matrix_secondary']=str(output)

In [ ]:
rf3 = Pipeline([('vect', CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', stop_words='english',max_df=0.85)),
       ('tfidf', TfidfTransformer()),
       ('clf', RandomForestClassifier(oob_score=True,n_jobs=-1,random_state=42,n_estimators=500))])
rf3.fit(X_train_final, y_train_final_tertiary)
y_pred= rf3.predict(X_test_final)
result['original_cat_tertiary']=y_test_final_tertiary
result['prediction_cat_tertiary']=y_pred
result['prediction_cat_tertiary_confscore']=rf3.predict_proba(X_test_final).max()
output={'accuracy':accuracy_score(y_pred,y_test_final_tertiary),'precision_score':precision_score(y_pred,y_test_final_tertiary,average='macro'),'recall_score':recall_score(y_pred,y_test_final_tertiary,average='macro')
,'f1_score':f1_score(y_pred,y_test_final_tertiary,average='macro')}
result['confusion_matrix_tertiary']=str(output)

In [32]:
import pickle
import datetime
# save the model to disk
filename_primary= 'finalized_model_rf_primary_{}_allprod.sav'.format(datetime.datetime.now())
filename_secondary= 'finalized_model_rf_secondary_{}_allprod.sav'.format(datetime.datetime.now())
filename_tertiary= 'finalized_model_rf_tertiary_{}_allprod.sav'.format(datetime.datetime.now())
pickle.dump(rf1, open(filename_primary, 'wb'))
pickle.dump(rf2, open(filename_secondary, 'wb'))
pickle.dump(rf3, open(filename_tertiary, 'wb'))


NameError: name 'rf' is not defined

In [ ]:
result

In [ ]:

misclassifications_primary= result.loc[result['original_cat_primary']!=result['prediction_cat_primary']]

In [ ]:
misclassifications_primary.sort_values(by=['Item','Description','establishment_type'], ascending=True).head(270)

In [ ]:
misclassifications_primary.to_csv('/Users/jghosh2/Documents/my-notebook/Tax_ml_poc/output/misclassifications_primary.csv')

In [ ]:
misclassifications_primary[misclassifications_primary['original_cat_primary']=='CAT_TPP_BATH_GELS,776']

"{'accuracy': 0.8146149522203485, 'precision_score': 0.6027551606729535, 'recall_score': 0.7097466761674291, 'f1_score': 0.6327463859326979}"

saving model to local

In [ ]:
import datetime
print("Time :{} result :{}".format(datetime.datetime.now(),result['confusion_matrix_primary'][0]))

Time :2022-01-10 13:30:26.356610 result :{'accuracy': 0.8185367800039901, 'precision_score': 0.5848761715254991, 'recall_score': 0.672646577840006, 'f1_score': 0.6100045958278678}

Time :2022-01-10 13:39:45.378508 result :{'accuracy': 0.8167982443640095, 'precision_score': 0.5828179565369508, 'recall_score': 0.694053097419111, 'f1_score': 0.6153519093480956}

Time :2022-01-10 14:04:13.438304 result :{'accuracy': 0.8207028244079003, 'precision_score': 0.598134322659592, 'recall_score': 0.7163954355735384, 'f1_score': 0.6336420913159138}

Time :2022-01-10 16:15:05.965823 result :{'accuracy': 0.8192777951948015, 'precision_score': 0.58422004758596, 'recall_score': 0.7013460556124523, 'f1_score': 0.6171140663609939}

Time :2022-01-10 16:30:28.661864 result :{'accuracy': 0.8203323168124946, 'precision_score': 0.5828415707142136, 'recall_score': 0.6922552428178115, 'f1_score': 0.6158350834301465}

Time :2022-01-10 17:13:56.042353 result :{'accuracy': 0.8250634137999829, 'precision_score': 0.6017204348512221, 'recall_score': 0.7016768888896173, 'f1_score': 0.6292577552019242}

Time :2022-01-10 17:45:33.576796 result :{'accuracy': 0.8197338045429932, 'precision_score': 0.6097855795962889, 'recall_score': 0.6989103740558321, 'f1_score': 0.6339990438994022}

Time :2022-01-10 18:00:59.932664 result :{'accuracy': 0.8222988571265711, 'precision_score': 0.6040505715950665, 'recall_score': 0.7041889895331753, 'f1_score': 0.630361835607479}

Time :2022-01-10 18:50:01.397963 result :{'accuracy': 0.8222988571265711, 'precision_score': 0.6040505715950665, 'recall_score': 0.7041889895331753, 'f1_score': 0.630361835607479}

{'accuracy': 0.8198348463305261, 'precision_score': 0.5943177708372124, 'recall_score': 0.6808857187011089, 'f1_score': 0.6211879572499776}

{'accuracy': 0.8341265995952917, 'precision_score': 0.6140457607875881, 'recall_score': 0.7160461281960248, 'f1_score': 0.6411403224021243}

{'accuracy': 0.8424202696155271, 'precision_score': 0.63494046605141, 'recall_score': 0.750621724728942, 'f1_score': 0.66790481231777}
n_estimator-100

{'accuracy': 0.8435317924017443, 'precision_score': 0.6366498170102253, 'recall_score': 0.7501320433368587, 'f1_score': 0.6679389982024275}----200

{'accuracy': 0.8443013081768176, 'precision_score': 0.6519943872650363, 'recall_score': 0.7635078463184546, 'f1_score': 0.6826975082666656}---300

## Regression test

In [13]:
import pickle
import os

In [2]:
primary_model_path = '/Users/ssingh355/code/taxml/data/final_model_2/primary'

In [16]:
# load the model from disk
primary_loaded_model = pickle.load(open(os.path.join(primary_model_path,'finalized_model.sav'), 'rb'))
# result = loaded_model.score(X_test, Y_test)
result = primary_loaded_model.score(df2['cleanText'], df2['primary'])
print(result)

0.9630694065239258


In [18]:
secondary_model_path = '/Users/ssingh355/code/taxml/data/final_model_2/secondary'

In [19]:
# load the model from disk
secondary_loaded_model = pickle.load(open(os.path.join(secondary_model_path,'finalized_model.sav'), 'rb'))
# result = loaded_model.score(X_test, Y_test)
result = secondary_loaded_model.score(df2['cleanText'], df2['secondary'])
print(result)

0.9425204770800403


## End of code flow - not consider below

In [ ]:
def split_cat(category):
    category_list=category.split(',')
    if len(category_list)==1:      
        primary_category=category_list[0]
        secondary_category=category_list[0]
    if len(category_list)==2:      
        primary_category=category_list[0]
        if category_list[1] in ['TEMP_HEATED','TEMP_COLD','TEMP_UNHEATED']:
              secondary_category=category_list[0]
        else:
              secondary_category=category_list[1]
    if len(category_list)>2:      
        primary_category=category_list[0]
        secondary_category=category_list[1]
    else:
        pass
        
    return primary_category,secondary_category
def split_int(integer):
    str_int=str(integer)
    integer_list=str_int.split(',')
    if len(integer_list)==1:      
        primary_integer=integer_list[0]
        secondary_integer=integer_list[0]
    if len(integer_list)==2:      
        primary_integer=integer_list[0]
        if integer_list[1]=='1':
            secondary_integer=integer_list[0]
        else:
            secondary_integer=integer_list[1]
    if len(integer_list)>2:      
        primary_integer=integer_list[0]
        secondary_integer=integer_list[1]
    else:
         pass
        
    return primary_integer,secondary_integer
def combine(category,integer):
    return category+","+integer

In [ ]:
df['primary_cat']=df['Agent Corrected CAT Name'].map(lambda x:split_cat(x)[0])
df['secondary_cat']=df['Agent Corrected CAT Name'].map(lambda x:split_cat(x)[1])
df['primary_int']=df['Agent Corrected Integer'].map(lambda x:split_int(x)[0])
df['secondary_int']=df['Agent Corrected Integer'].map(lambda x:split_int(x)[1])
df['primary_int_prediction']=df['Integer'].map(lambda x:split_int(x)[0])
df['primary']=df[['primary_cat','primary_int']].apply(lambda x:combine(*x),axis=1)
df['secondary']=df[['secondary_cat','secondary_int']].apply(lambda x:combine(*x),axis=1)

In [ ]:
df.to_csv('TaxML-CICD - Prod_Data_after_preprocess.csv')

In [ ]:
df

In [ ]:
df_test = df.loc[df['Inetger_ValidationScore[0-100]'] == 0]

In [ ]:
len(df_test)

In [ ]:
df2=df_test[df_test['primary_int_prediction']==df_test['primary_int']]
len(df2)#no of correct prediction

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint
from time import time
import logging
rf = Pipeline([('vect', CountVectorizer(strip_accents='ascii', token_pattern=u'(?ui)\\b\\w*[a-z]+\\w*\\b', lowercase=True, stop_words='english')),
       ('tfidf', TfidfTransformer(use_idf=True)),
       ('clf', RandomForestClassifier()),
      ])
parameters = {
    "vect__max_df": (0.5, 0.75, 1.0),
    # 'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigramslf__
     "clf__bootstrap":[True, False],
     "clf__max_depth":[10, 50, 100,500, None],
     "clf__max_features":['auto', 'sqrt'],
     "clf__min_samples_leaf":[1,2,4],
     "clf__min_samples_split":[2,5,10],
     "clf__n_estimators":[400,600,800],
     "clf__random_state":[3]

}

RandomizedSearch = RandomizedSearchCV(rf,
                          parameters, 
                          cv=5,
                          verbose=1, 
                          n_jobs=-1)

t0 = time()
rf_best_model = RandomizedSearch.fit(X_train_final, y_train_final_primary)
print("done in %0.3fs" % (time() - t0))
#print()
print("Best score: %0.3f" % rf_best_model.best_score_)
print("Best parameters set:")
best_parameters = rf_best_model.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [ ]:
best_parameters

169000 total data b4 traing + 403 rows
exp01:169000+403 rows
exp02:169000+727 rows
